In [4]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': '7589380f3203',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '67ETAy1pSF2UL6kIPc3vEA',
 'version': {'number': '8.3.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56',
  'build_date': '2022-07-23T19:30:09.227964828Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [6]:
df = pd.read_csv('kinoloisk4.csv', sep=";")
df.head()

,Unnamed: 0,link,eng_name,rus_name,year,duration,country,genre,director_name,rating
0,0,https://www.kinopoisk.ru/film/1249172/,NaN,Последний богатырь: Корень зла,2020.0,121.0,Россия,фэнтези,Дмитрий Дьяченко,NaN
1,1,https://www.kinopoisk.ru/film/8366/,The Thing,Нечто,1982.0,109.0,США,ужасы,Джон Карпентер,7.9
2,2,https://www.kinopoisk.ru/film/79850/,NaN,Ночной дозор,2004.0,115.0,Россия,фэнтези,Тимур Бекмамбетов,NaN
3,3,https://www.kinopoisk.ru/film/8151/,Interview with the Vampire: The Vampire Chroni...,Интервью с вампиром,1994.0,123.0,США,драма,Нил Джордан,8.0
4,4,https://www.kinopoisk.ru/film/4322004/,NaN,Плакать нельзя,2022.0,125.0,Россия,триллер,Наталья Назарова,NaN


In [7]:
df[['year', 'duration']] = df[['year', 'duration']].fillna(0)
df[['year', 'duration']] = df[['year', 'duration']].astype(int)
df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']] = df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']].fillna("nan")
df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']] = df[['link', 'eng_name', 'rus_name', 'country', 'genre', 'director_name']].astype(str)
df[['rating']] = df[['rating']].fillna(-1.0)

In [8]:
mappings = {
        "properties": {
            "link": {"type": "keyword"},
            "eng_name": {"type": "text", "analyzer": "english"},
            "rus_name": {"type": "text", "analyzer": "russian"},
            "year": {"type": "integer"},
            "duration": {"type": "integer"},
            "country": {"type": "text", "analyzer": "russian"},
            "genre": {"type": "text", "analyzer": "russian"},
            "director_name": {"type": "text", "analyzer": "russian"},
            "rating": {"type": "float"}
    }
}

es.indices.create(index="movies", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

In [9]:
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "movies",
            "_id": i,
            "_source": {        
                "link": row["link"],
                "eng_name": row["eng_name"],
                "rus_name": row["rus_name"],
                "year": row["year"],
                "duration": row["duration"],
                "country": row["country"],
                "genre": row["genre"],
                "director_name": row["director_name"],
                "rating": row["rating"],
            }
        }
    )
bulk(es, bulk_data)

(1000, [])

In [10]:
es.indices.refresh(index="movies")
es.cat.count(index="movies", format="json")

ListApiResponse([{'epoch': '1668855708', 'timestamp': '11:01:48', 'count': '1000'}])

In [43]:
q1 = es.search(
    index="movies",
    body= {
        "query": {
            "multi_match": {
                "query": 'богатырь',
                "fields": [
                    "eng_name",
                    "rus_name",
                    "country",
                    "genre",
                    "director_name"
                ],
                "type" : "cross_fields",
                "analyzer" : "russian",
                "prefix_length" : "3"
            }
        }
    }
)
q1['hits']

C:\Users\danil\AppData\Local\Temp\ipykernel_5016\1574783358.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q1 = es.search(


{'total': {'value': 5, 'relation': 'eq'},
 'max_score': 5.250845,
 'hits': [{'_index': 'movies',
   '_id': '416',
   '_score': 5.250845,
   '_source': {'link': 'https://www.kinopoisk.ru/film/976642/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь',
    'year': 2017,
    'duration': 114,
    'country': 'Россия',
    'genre': 'комедия',
    'director_name': 'Дмитрий Дьяченко',
    'rating': -1.0}},
  {'_index': 'movies',
   '_id': '28',
   '_score': 4.3705106,
   '_source': {'link': 'https://www.kinopoisk.ru/film/4419497/',
    'eng_name': 'nan',
    'rus_name': 'Три богатыря и Конь на троне',
    'year': 2021,
    'duration': 202,
    'country': 'Россия',
    'genre': 'мультфильм',
    'director_name': 'Дарина Шмидт',
    'rating': 7.4}},
  {'_index': 'movies',
   '_id': '0',
   '_score': 3.742979,
   '_source': {'link': 'https://www.kinopoisk.ru/film/1249172/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь: Корень зла',
    'year': 2020,
    'duration': 121,
 

In [45]:
q2 = es.search(
    index = "movies",
    body = {
      "query": {
        "wildcard": {
          "rus_name": {
            "value": '*' + "богаты**",
            "boost": 1.0,
            "rewrite": "constant_score"
          }
        },
      }
    }
)
q2['hits']

C:\Users\danil\AppData\Local\Temp\ipykernel_5016\2226648462.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q2 = es.search(


{'total': {'value': 5, 'relation': 'eq'},
 'max_score': 1.0,
 'hits': [{'_index': 'movies',
   '_id': '0',
   '_score': 1.0,
   '_source': {'link': 'https://www.kinopoisk.ru/film/1249172/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь: Корень зла',
    'year': 2020,
    'duration': 121,
    'country': 'Россия',
    'genre': 'фэнтези',
    'director_name': 'Дмитрий Дьяченко',
    'rating': -1.0}},
  {'_index': 'movies',
   '_id': '28',
   '_score': 1.0,
   '_source': {'link': 'https://www.kinopoisk.ru/film/4419497/',
    'eng_name': 'nan',
    'rus_name': 'Три богатыря и Конь на троне',
    'year': 2021,
    'duration': 202,
    'country': 'Россия',
    'genre': 'мультфильм',
    'director_name': 'Дарина Шмидт',
    'rating': 7.4}},
  {'_index': 'movies',
   '_id': '254',
   '_score': 1.0,
   '_source': {'link': 'https://www.kinopoisk.ru/film/1272246/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь: Посланник Тьмы',
    'year': 2021,
    'duration': 108,
    '

In [46]:
q3 = es.search(
    index =  'movies',
    body = {
      "query": {
        "bool": {
          "should": [
            {
              "wildcard": {
                "rus_name": {
                  "value": '*' + 'багаты**',
                  "boost": 1.0,
                  "rewrite": "constant_score"
                }
              },
            },
            {
              "match": {
                "rus_name": {
                  "query": "багатырь",
                  "operator": 'or',
                  "fuzziness": 'auto'
                }
              }
            }
          ]
        }
      }
    }
)
q3['hits']

C:\Users\danil\AppData\Local\Temp\ipykernel_5016\156127524.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  q3 = es.search(


{'total': {'value': 5, 'relation': 'eq'},
 'max_score': 4.5007243,
 'hits': [{'_index': 'movies',
   '_id': '416',
   '_score': 4.5007243,
   '_source': {'link': 'https://www.kinopoisk.ru/film/976642/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь',
    'year': 2017,
    'duration': 114,
    'country': 'Россия',
    'genre': 'комедия',
    'director_name': 'Дмитрий Дьяченко',
    'rating': -1.0}},
  {'_index': 'movies',
   '_id': '28',
   '_score': 3.746152,
   '_source': {'link': 'https://www.kinopoisk.ru/film/4419497/',
    'eng_name': 'nan',
    'rus_name': 'Три богатыря и Конь на троне',
    'year': 2021,
    'duration': 202,
    'country': 'Россия',
    'genre': 'мультфильм',
    'director_name': 'Дарина Шмидт',
    'rating': 7.4}},
  {'_index': 'movies',
   '_id': '0',
   '_score': 3.2082677,
   '_source': {'link': 'https://www.kinopoisk.ru/film/1249172/',
    'eng_name': 'nan',
    'rus_name': 'Последний богатырь: Корень зла',
    'year': 2020,
    'duration': 121,

In [ ]:
# resp = es.search(
#     index="movies",
#     body= {
#         "query": {
#             "function_score": {
#                 "query": {
#                     "multi_match": {
#                         "query": 'богатырь',
#                         "fields": [
#                             "eng_name",
#                             "rus_name",
#                             "country",
#                             "genre",
#                             "director_name"
#                         ],
#                         "type" : "cross_fields",
#                         "analyzer" : "russian",
#                         "prefix_length" : "3"
#                     },
#                 },
#                 "functions": [
#                     {
#                         "script_score": {
#                             "script": {
#                             "source": "Math.log(2 + doc['my-int'].value)"
#                             }
#                         }
#                     }
#                 ],
#             }
#         }
#     }
# )
# resp['hits']